# Model Explainability with SHAP: A Guide to Those Who Are Serious About Machine Learning
## SUBTITLE TODO
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@iriser?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Irina Iriser</a>
        on 
        <a href=https://www.pexels.com/photo/blue-and-red-jellyfish-artwork-1086583/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels''></a>
    </strong>
</figcaption>

# Setup

In [1]:
import logging
import time
import warnings

import catboost as cb
import datatable as dt
import joblib
import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.compose import (
    ColumnTransformer,
    make_column_selector,
    make_column_transformer,
)
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%d-%b-%y %H:%M:%S", level=logging.INFO
)
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")
pd.set_option("float_format", "{:.5f}".format)

In [4]:
# For regression
diamonds = sns.load_dataset("diamonds")

X, y = diamonds.drop("price", axis=1), diamonds[["price"]].values.flatten()

# Encode cats
oe = OrdinalEncoder()
cats = X.select_dtypes(exclude=np.number).columns.tolist()
X.loc[:, cats] = oe.fit_transform(X[cats])

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.15, random_state=1121218
)

In [3]:
# For classification - HIDE
diamonds = sns.load_dataset("diamonds")

X, y = diamonds.drop("cut", axis=1), diamonds[["cut"]].values.flatten()

# Encode cats
oe = OrdinalEncoder()
cats = X.select_dtypes(exclude=np.number).columns.tolist()
X.loc[:, cats] = oe.fit_transform(X[cats])

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.15, random_state=1121218
)

# Motivation

Today, you can't just come up to your boss and say, "Here is my best model. Let's put it into production and be happy!". No, it doesn't work that way now. Companies and businesses are being picky over the adoption of AI solutions because of their "black box" nature. They **demand** explainability. 

If ML specialists are coming up with tools to understand and explain the tools *they* created, the concerns and suspicions of non-technical folks is entirely justified. One of those tools introduced a few years ago is SHAP. It has the ability to break down mechanics of any machine learning model and deep neural net to make them understandable to anyone. TODO 

Today, we will learn how exactly SHAP works and how you can use it for classical ML tasks in your own practice. 

# What is SHAP and Shapley values?

SHAP (SHapley Additive exPlanations) is a Python package based on the 2016 NIPS paper about SHAP values. The premise of this paper and Shapley values comes from approaches in game theory. 

One of the questions often posed in games is that in a group of *n* players with different skillsets, how do we divide a prize in a way that everyone gets a fair share based on their skillset? Depending on the number of players, their time of joining the game and their different contributions to the outcome, this type of calculation can become horribly complex.

But what does game theory have to do with machine learning? Well, we could reframe the above question so that it becomes "Given a prediction, how do we most accurately measure each feature's contribution?" Yes, it is kinda like asking feature importances of a model but the answer the Shapley values give is much more sophisticated. 

Specifically, Shapley values can help you in:
1. *Global model interpretability* - imagine you work for a bank and you build a classification model for loan applications. Your manager wants you to explain what (and how) different factors influence the decisions of your model. Using SHAP values, you can give a concrete answer with details of which features lead to more loans and which features lead to more rejections. You make your manager happy because now, he can draw up basic guidelines for future customers of the bank to increase their chances of getting a loan. More loans - more money for the bank.

TODO - show a sample plot.

2. *Local interpretability* - your model rejects one of the applications submitted to the bank a few days ago. The customer claims he followed all the guidelines and was sure to get a loan from your bank. Now, you are legally obligated to explain why your model rejected that particular candidate. Using Shapley values, every case can be analyzed on its own, without worrying about its connections to other samples in the data. In other words, you have local interpretability. You extract the Shapley values for the complaining customer and show them what parts of their application caused the rejection. You prove them wrong.

TODO - show a sample plot.

So, how do you calculate the mighty Shapley values? That's where we start using the SHAP package.

# Code

In [37]:
# Init/train
model = xgb.XGBRegressor(n_estimators=1000, tree_method="gpu_hist").fit(
    X_train, y_train
)

In [38]:
# Generate preds/score
preds = model.predict(X_valid)
rmse = mean_squared_error(y_valid, preds, squared=False)

In [39]:
rmse

573.9077249528166

In [40]:
# Create a tree explainer
xgb_explainer = shap.TreeExplainer(
    model, X_train, feature_names=X_train.columns.tolist()
)

In [41]:
%%time

# Shap values with tree explainer
shap_values = xgb_explainer.shap_values(X_train, y_train)

100%|===================| 45830/45849 [21:36<00:00]        

Wall time: 21min 37s


In [42]:
shap_values.shape

(45849, 9)

In [43]:
%%time

# Shap values with XGBoost core moedl
booster_xgb = model.get_booster()
shap_values_xgb = booster_xgb.predict(xgb.DMatrix(X_train, y_train), pred_contribs=True)

Wall time: 1.42 s


In [44]:
shap_values_xgb.shape

(45849, 10)

In [45]:
%%time

# SHAP interactions with XGB
interactions_xgb = booster_xgb.predict(
    xgb.DMatrix(X_train, y_train), pred_interactions=True
)

Wall time: 15.3 s


In [46]:
interactions_xgb.shape

(45849, 10, 10)